# Cell Differentiation GO analysis

This analysis is to support https://github.com/geneontology/go-ontology/issues/24390

The goal here is to explore

 - consistent population of terms in "cell differentiation" patterns in the ontology
 - consistency of annotation of terms at different levels
 
This is to help us decide on how to best patternize differentiation. Ideally:

 1. We should consistently populate terms
 2. We should consistently annotate terms


__NOTE FOR ONTOLOGY EDITORS__ This is a Jupyter notebook. You do not need to understand any of the code here.
The notebook form makes the generation of the stats here reproducible and transparent. Any member of the GO
software team can help you run this for yourself, or tweak settings.

Scroll to the bottom for the main results.



## Initial setup

Here we import the necessary libraries:
    
 1. OAK, for ontology querying (labels and logical definitions)
 2. go_library, for annotation querying

In [41]:
from oaklib.selector import get_implementation_from_shorthand
from oaklib.datamodels.vocabulary import IS_A, PART_OF

In [14]:
oi = get_implementation_from_shorthand("sqlite:obo:go")

In [1]:
from go_library.datamodel.amigo_solr_api import AmigoSolrAPI
from go_library.amigo_solr.engine import create_handle
from go_library.datamodel.amigo_solr import Annotation, Bioentity

In [2]:
amigo = create_handle()

In [3]:
HUMAN = 'NCBITaxon:9606'

## IBA summaries

We write a convenience method to fetch IBA stats from AmiGO.

The intuition here is that terms annotated with IBA have been deemed to be trustworthy of propagation

In [76]:
def get_IBA_summary(term, taxon):
    """
    Returns: (total_anns, anns_to_IBA)
    """
    solr_params = {'facet': 'on',
                   'facet.mincount': 1,
                   'facet.field': ['evidence_type'],
                   'rows': 0}
    results = amigo.query_engine.search(Annotation,
                                                     solr_params=solr_params,
                                                     isa_partof_closure=term,
                                                     taxon=taxon)
    return results.num_found, results.facet_counts['evidence_type'].get('IBA', 0)


# test with an example term
get_IBA_summary("GO:0060911", HUMAN)
    

(14, 0)

## Convenience function to fetch genes

In [8]:
def get_genes(term, taxon):
    results = amigo.query_engine.search(Bioentity,
                                                     solr_params={'fl':'id', 'rows': 10000},
                                                     isa_partof_closure=term,
                                                     taxon=taxon)
    return [e.id for e in results.items]

get_genes("GO:0060911", HUMAN)

['UniProtKB:P56704',
 'UniProtKB:Q9H6I2',
 'UniProtKB:P19544',
 'UniProtKB:O15119',
 'UniProtKB:P61371',
 'UniProtKB:P05556',
 'UniProtKB:Q04771',
 'UniProtKB:O95935',
 'UniProtKB:Q9H1J5',
 'UniProtKB:P52952',
 'UniProtKB:Q9BRJ9',
 'UniProtKB:Q6N022',
 'UniProtKB:Q99593']

## Upset plots

These are superior to Venn diagrams in that they can show intersection between arbitrary numbers of sets.

We will use these to explore how often the same gene is populated to different terms

In [77]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetJSWidget
import pandas as pd

In [78]:
w = UpSetJSWidget[str]()

In [64]:
def compare_set(terms, taxon):
    results = {}
    for term in terms:
        label = oi.label(term)
        genes = get_genes(term, taxon)
        results[label] = genes
    return w.from_dict(results)


In [79]:
compare_set(["GO:0060911", "GO:0060912"], HUMAN)

UpSetJSWidget(value=None, combinations=[UpSetSetIntersection(name=cardiac cell fate commitment, sets={'cardiac…

## Cell Differentiation logical definitions

Here we fetch all cell differentiation descendants over is-a and part-of, and get all logical
definitions that reference a cell type

In [42]:
CELL_DIFFERENTIATION = "GO:0030154"
terms = list(oi.descendants(CELL_DIFFERENTIATION, predicates=[IS_A, PART_OF]))

In [43]:
ldefs = list(oi.logical_definitions(terms))

In [44]:
# check how many
len(ldefs)

589

In [80]:
# index by cell type
from collections import defaultdict

ldef_by_cell = defaultdict(list)
for ldef in ldefs:
    if len(ldef.restrictions) != 1:
        continue
    cell = ldef.restrictions[0].fillerId
    if not cell.startswith("CL:") and not cell.startswith("PO:") and not cell.startswith("FAO:"):
        continue
    ldef_by_cell[cell].append(ldef)
print(len(ldef_by_cell))

262


In [50]:
# filter to only cell types that have >1 associated logical definition
ldef_by_cell = {k: v for k, v in ldef_by_cell.items() if len(v) > 1}

## Summarize Logical Definitions indexed by Cell Type

 - **TODO** make this prettier
 - **TODO** filter to get only differentiation DPs (diff, fate commitment, specification, maturation, ...)

In [82]:
for cell, cell_ldefs in ldef_by_cell.items():
    genus_terms = [ldef.genusIds for ldef in cell_ldefs]
    print(f"  {cell} {oi.label(cell)} : {len(cell_ldefs)} -- {[oi.label(g[0]) for g in genus_terms]}")

  CL:0000726 chlamydospore : 1 -- ['asexual sporulation resulting in formation of a cellular spore']
  CL:0000540 neuron : 3 -- ['cell migration', 'cell recognition', 'developmental maturation']
  CL:0000623 natural killer cell : 1 -- ['cell differentiation']
  CL:2000020 inner cell mass cell : 2 -- ['cell differentiation', 'cell fate commitment']
  CL:1000274 trophectodermal cell : 3 -- ['cell differentiation', 'cell fate commitment', 'anatomical structure morphogenesis']
  CL:0000814 mature NK T cell : 2 -- ['cell differentiation', 'cell fate commitment']
  CL:0000115 endothelial cell : 3 -- ['developmental process', 'anatomical structure morphogenesis', 'cell differentiation']
  CL:0000819 B-1 B cell : 2 -- ['cell differentiation', 'cell fate commitment']
  CL:2000021 sebaceous gland cell : 1 -- ['cell differentiation']
  CL:0000062 osteoblast : 2 -- ['cell fate commitment', 'developmental process']
  CL:0000138 chondrocyte : 3 -- ['cell differentiation', 'developmental process', 'a

  CL:0000562 nucleate erythrocyte : 1 -- ['developmental process']
  CL:0000034 stem cell : 5 -- ['cell differentiation', 'developmental process', 'cell fate commitment', 'cell fate specification', 'cell fate determination']
  CL:0000851 neuromast mantle cell : 1 -- ['cell differentiation']
  CL:0000852 neuromast supporting cell : 1 -- ['cell differentiation']
  CL:2000031 lateral line ganglion neuron : 1 -- ['cell differentiation']
  CL:1000050 lateral line nerve glial cell : 3 -- ['cell differentiation', 'cell migration', 'cell development']
  CL:2000035 anterior lateral line neuromast mantle cell : 1 -- ['cell differentiation']
  CL:2000036 anterior lateral line neuromast supporting cell : 1 -- ['cell differentiation']
  CL:1000239 anterior lateral line nerve glial cell : 2 -- ['cell differentiation', 'cell development']
  CL:2000038 posterior lateral line neuromast mantle cell : 1 -- ['cell differentiation']
  CL:2000039 posterior lateral line neuromast supporting cell : 1 -- ['cel

## Cell Type Summarization function

This creates a function that will

 - summarize all GO terms with logical defs that reference that cell
 - give basic annotation stats
 - draw an upset plot showing how genes are distributed across those terms
 
 TODO: draw a visualization of how the terms relate

In [74]:
def cell_summary(cell, taxon):
    cell_label = oi.label(cell)
    print(f"SUMMARY FOR {cell} {cell_label}")
    go_terms = [ldef.definedClassId for ldef in ldef_by_cell[cell]]
    for term in go_terms:
        anns, ibas = get_IBA_summary(term, taxon)
        print(f" * {term} {oi.label(term)} IBAs: {ibas} Anns: {anns}")
    return compare_set(go_terms, taxon)
    

## EXAMPLE: Stem cell (human)

In [83]:
cell_summary("CL:0000034", HUMAN)

SUMMARY FOR CL:0000034 stem cell
 * GO:0048863 stem cell differentiation IBAs: 27 Anns: 212
 * GO:0048864 stem cell development IBAs: 27 Anns: 101
 * GO:0048865 stem cell fate commitment IBAs: 0 Anns: 10
 * GO:0048866 stem cell fate specification IBAs: 0 Anns: 6
 * GO:0048867 stem cell fate determination IBAs: 0 Anns: 0


UpSetJSWidget(value=None, combinations=[UpSetSetIntersection(name=stem cell differentiation, sets={'stem cell …

### observations

note the majority of genes are on the less leafy development/differentiation nodes.

there is a lot of overlap between these

TODO: 

In [84]:
MOUSE = "NCBITaxon:10090"
cell_summary("CL:0000034", MOUSE)

SUMMARY FOR CL:0000034 stem cell
 * GO:0048863 stem cell differentiation IBAs: 27 Anns: 339
 * GO:0048864 stem cell development IBAs: 27 Anns: 166
 * GO:0048865 stem cell fate commitment IBAs: 0 Anns: 10
 * GO:0048866 stem cell fate specification IBAs: 0 Anns: 5
 * GO:0048867 stem cell fate determination IBAs: 0 Anns: 0


UpSetJSWidget(value=None, combinations=[UpSetSetIntersection(name=stem cell differentiation, sets={'stem cell …